In [2]:
import pandas as pd
from prefect import task, Flow, Parameter

import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [54]:
@task
def fetch_data():
    # Read the wine-quality csv file from the URL
    csv_url =\
        'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
    try:
        data = pd.read_csv(csv_url, sep=';')
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e)
    return data


@task
def train_model(
                data: pd.DataFrame,
                in_alpha: float, 
                in_l1_ratio: float,
                mlflow_experiment_id: int):
    
    mlflow.set_tracking_uri("http://mlflow.datarevenue.com:5000")
    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        
        return rmse, mae, r2

    
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    # Set default values if no alpha is provided
    if float(in_alpha) is None:
        alpha = 0.5
    else:
        alpha = float(in_alpha)

    # Set default values if no l1_ratio is provided
    if float(in_l1_ratio) is None:
        l1_ratio = 0.5
    else:
        l1_ratio = float(in_l1_ratio)

    # Useful for multiple runs (only doing one run in this sample notebook)    
    with mlflow.start_run(experiment_id=mlflow_experiment_id):
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, "model")

In [3]:
from prefect.run_configs import KubernetesRun
from prefect.storage import S3


In [1]:
def main():
    custom_confs = {
        "run_config": KubernetesRun(
            image="drtools/prefect:python-job-executor", 
            labels=["stage"],
            service_account_name=
        ),   
        "storage": S3(bucket="dr-prefect"),

    } 
    with Flow("wine-classifier", **custom_confs, ) as flow:
        in_alpha = Parameter("in_alpha", default=0.5)
        in_l1_ratio = Parameter("in_l1_ratio", default=0.5)

        data = fetch_data()
        train_model(data, mlflow_experiment_id=1, in_alpha=in_alpha, in_l1_ratio=in_l1_ratio)


    flow.register(project_name="mlops-demo")
    #flow.run()

In [63]:
main()

Result check: OK
[2021-02-06 21:42:06-0300] INFO - prefect.S3 | Uploading wine-classifier/2021-02-07t00-42-06-813848-00-00 to dr-prefect


INFO:prefect.S3:Uploading wine-classifier/2021-02-07t00-42-06-813848-00-00 to dr-prefect


Flow URL: http://localhost:8080/default/flow/cbdea589-6b7f-4841-b533-1a9767e1e401
 └── ID: 34143d15-5710-4c2d-a42b-ae73ccfb5013
 └── Project: mlops-demo
 └── Labels: ['stage']


In [ ]:
Kubee